In [53]:
import http.client
import json
import psycopg2

# Configuración de la conexión y los encabezados
conn_api = http.client.HTTPSConnection("v2.nba.api-sports.io")
headers = {
    'x-rapidapi-host': "v2.nba.api-sports.io",
    'x-rapidapi-key': "3f41540dfbba32863f7981122084859b"
}

# Lista para almacenar los datos de los jugadores
players_data = []

# Bucle para obtener datos de los jugadores con IDs del 1 al 10
for player_id in range(100, 102):
    # Crear y enviar la solicitud
    endpoint = f"/players?id={player_id}"
    conn_api.request("GET", endpoint, headers=headers)

    # Obtener la respuesta y decodificarla
    res = conn_api.getresponse()
    data = res.read()
    player_data = json.loads(data.decode("utf-8"))

    # Verificar la respuesta y agregar los datos del jugador a la lista
    if 'response' in player_data and player_data['response']:
        players_data.append(player_data['response'][0])

# Datos de conexión a Redshift
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
dbname = "data-engineer-database"
user = "lucasgfara_coderhouse"
password = "Bs1H2V5S4C"
port = "5439"

try:
    # Conexión a Redshift y creación de tabla
    conn_redshift = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password,
        port=port
    )
    cur = conn_redshift.cursor()

    # Creación de la tabla en Redshift si no existe
    cur.execute("""
        CREATE TABLE IF NOT EXISTS nba_players (
            id INT PRIMARY KEY,
            firstname VARCHAR(50),
            lastname VARCHAR(50),
            birth DATE,
            nba_start INT,
            nba_pro INT,
            height VARCHAR(10),
            weight VARCHAR(10),
            college VARCHAR(100),
            affiliation VARCHAR(100),
            leagues TEXT
        )
    """)

    # Función para convertir datos de jugador en tupla
    def player_to_tuple(player):
      birth = player['birth']['date'] if player['birth']['date'] else None
      nba_start = player['nba']['start'] if player['nba']['start'] else None
      nba_pro = player['nba']['pro'] if player['nba']['pro'] else None
      leagues = json.dumps(player['leagues'])  # Convertir el diccionario de ligas a una cadena JSON
      return (
          player['id'], player['firstname'], player['lastname'], birth,
          nba_start, nba_pro, player['height']['inches'] if player['height'] else None,  # Asumiendo que 'height' tiene una clave 'inches'
          player['weight']['pounds'] if player['weight'] else None,  # Asumiendo que 'weight' tiene una clave 'pounds'
          player['college'], player['affiliation'], leagues
      )


    # Insertar datos obtenidos de la API en la tabla
    for player in players_data:
        player_tuple = player_to_tuple(player)
        cur.execute("""
            INSERT INTO nba_players (id, firstname, lastname, birth, nba_start, nba_pro, height, weight, college, affiliation, leagues)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """, player_tuple)

    # Confirmar los cambios y cerrar la conexión
    conn_redshift.commit()
    print("Tabla creada y datos insertados en Redshift con éxito.")

except psycopg2.Error as e:
    print("Error al conectar a Redshift o al insertar datos:", e)

finally:
    if cur:
        cur.close()
    if conn_redshift:
        conn_redshift.close()


Tabla creada y datos insertados en Redshift con éxito.


In [46]:
import http.client
import json
import pandas as pd
import psycopg2

# Configuración de la conexión y los encabezados
conn_api = http.client.HTTPSConnection("v2.nba.api-sports.io")
headers = {
    'x-rapidapi-host': "v2.nba.api-sports.io",
    'x-rapidapi-key': "3f41540dfbba32863f7981122084859b"
}

# Lista para almacenar los datos de los jugadores
players_data = []

# Bucle para obtener datos de los jugadores con IDs del 1 al 100
for player_id in range(90, 100):
    # Crear y enviar la solicitud
    endpoint = f"/players?id={player_id}"
    conn_api.request("GET", endpoint, headers=headers)

    # Obtener la respuesta y decodificarla
    res = conn_api.getresponse()
    data = res.read()
    player_data = json.loads(data.decode("utf-8"))

    # Verificar la respuesta y agregar los datos del jugador a la lista
    if 'response' in player_data and player_data['response']:
        players_data.append(player_data['response'][0])

# Crear DataFrame a partir de los datos de los jugadores
df = pd.DataFrame(players_data)

# Eliminar filas con valores nulos en cualquier columna
df = df.dropna()

# Renombrar las columnas college y affiliation
df = df.rename(columns={'college': 'university', 'affiliation': 'affiliate team'})

# Eliminar filas duplicadas basadas en todas las columnas del DataFrame
df.drop_duplicates(inplace=True)

# Concatenar firstname y lastname en una nueva columna 'full_name'
df['full_name'] = df['firstname'] + ' ' + df['lastname']

# Datos de conexión a Redshift
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
dbname = "data-engineer-database"
user = "lucasgfara_coderhouse"
password = "Bs1H2V5S4C"
port = "5439"

try:
    # Conexión a Redshift y creación de tabla
    conn_redshift = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password,
        port=port
    )
    cur = conn_redshift.cursor()

    # Creación de la tabla en Redshift si no existe
    cur.execute("""
        CREATE TABLE IF NOT EXISTS nba_players_new (
            id INT PRIMARY KEY,
            full_name VARCHAR(100),
            birth DATE,
            nba_start INT,
            nba_pro INT,
            height VARCHAR(10),
            weight VARCHAR(10),
            university VARCHAR(100),
            "affiliate team" VARCHAR(100),
            leagues TEXT
        )
    """)

    # Insertar datos del DataFrame en la tabla de Redshift
    for index, row in df.iterrows():
        cur.execute("""
            INSERT INTO nba_players_new (id, full_name, birth, nba_start, nba_pro, height, weight, university, "affiliate team", leagues)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """, (row['id'], row['full_name'], row['birth'], row['nba_start'], row['nba_pro'], row['height'], row['weight'], row['university'], row['affiliate team'], json.dumps(row['leagues'])))

    # Confirmar los cambios y cerrar la conexión
    conn_redshift.commit()
    print("Tabla creada y datos insertados en Redshift con éxito.")

except psycopg2.Error as e:
    print("Error al conectar a Redshift o al insertar datos:", e)

finally:
    if cur:
        cur.close()
    if conn_redshift:
        conn_redshift.close()


KeyError: 'firstname'

In [59]:
import http.client
import json
import pandas as pd
import psycopg2

# Configuración de la conexión y los encabezados
conn_api = http.client.HTTPSConnection("v2.nba.api-sports.io")
headers = {
    'x-rapidapi-host': "v2.nba.api-sports.io",
    'x-rapidapi-key': "3f41540dfbba32863f7981122084859b"
}

# Lista para almacenar los datos de los jugadores
players_data = []

# Bucle para obtener datos de los jugadores con IDs del 1 al 100
for player_id in range(1, 3):
    # Crear y enviar la solicitud
    endpoint = f"/players?id={player_id}"
    conn_api.request("GET", endpoint, headers=headers)

    # Obtener la respuesta y decodificarla
    res = conn_api.getresponse()
    data = res.read()
    player_data = json.loads(data.decode("utf-8"))

    # Verificar la respuesta y agregar los datos del jugador a la lista
    if 'response' in player_data and player_data['response']:
        players_data.append(player_data['response'][0])


# Crear DataFrame a partir de los datos de los jugadores
df = pd.DataFrame(players_data)
print(df)

# Eliminar filas con valores nulos en cualquier columna
df = df.dropna()

# Renombrar las columnas college y affiliation
df = df.rename(columns={'college': 'university', 'affiliation': 'affiliate team'})

# Eliminar filas duplicadas basadas en todas las columnas del DataFrame
df.drop_duplicates(inplace=True)

# Concatenar firstname y lastname en una nueva columna 'full_name'
# df['full_name'] = df['firstname'] + ' ' + df['lastname']

# Eliminar las columnas firstname y lastname del DataFrame
# df = df.drop(['firstname', 'lastname'], axis=1)

# Datos de conexión a Redshift
host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
dbname = "data-engineer-database"
user = "lucasgfara_coderhouse"
password = "Bs1H2V5S4C"
port = "5439"

try:
    # Conexión a Redshift y creación de tabla
    conn_redshift = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password,
        port=port
    )
    cur = conn_redshift.cursor()

    # Creación de la tabla en Redshift si no existe
    cur.execute("""
        CREATE TABLE IF NOT EXISTS nba_players_new (
            id INT PRIMARY KEY,
            birth DATE,
            nba_start INT,
            nba_pro INT,
            height VARCHAR(10),
            weight VARCHAR(10),
            university VARCHAR(100),
            "affiliate team" VARCHAR(100),
            leagues TEXT,
            full_name VARCHAR(100)
        )
    """)

    # Insertar datos del DataFrame en la tabla de Redshift
    for index, row in df.iterrows():
        cur.execute("""
            INSERT INTO nba_players_new (id, birth, nba_start, nba_pro, height, weight, university, "affiliate team", leagues, full_name)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """, (row['id'], row['birth'], row['nba_start'], row['nba_pro'], row['height'], row['weight'], row['university'], row['affiliate team'], json.dumps(row['leagues']), row['full_name']))

    # Confirmar los cambios y cerrar la conexión
    conn_redshift.commit()
    print("Tabla creada y datos insertados en Redshift con éxito.")

except psycopg2.Error as e:
    print("Error al conectar a Redshift o al insertar datos:", e)

finally:
    if cur:
        cur.close()
    if conn_redshift:
        conn_redshift.close()


Empty DataFrame
Columns: []
Index: []
Tabla creada y datos insertados en Redshift con éxito.
